# LangServeで遊ぶ

次のコマンドでWebアプリケーションを起動してからノートブックを実行していく。

```sh
poetry run task start
```

In [1]:
import requests
from datetime import datetime

base_url = "http://localhost:8000"

## とりあえずOpenAIでチャット

In [2]:
json = {
    "input": "日本の首都は？"
}
resp = requests.post(f"{base_url}/chat1/invoke", json=json)
resp.json()

{'output': {'content': '東京（とうきょう）です。',
  'additional_kwargs': {},
  'type': 'ai',
  'name': None,
  'id': None,
  'example': False},
 'callback_events': [],
 'metadata': {'run_id': 'c65ade24-93cd-49fb-a7ff-c7402e88dd25'}}

In [3]:
json = {
    "input": "日本の首都は？"
}
resp = requests.post(f"{base_url}/chat1/stream", json=json, stream=True)
[line for line in resp.iter_lines(decode_unicode=True)]

['event: metadata',
 'data: {"run_id": "40f36355-2c77-4758-8ee5-05cae44c0860"}',
 '',
 'event: data',
 'data: {"content":"","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"東","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"京","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"です","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"。","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: end',
 '']

## `RunnablePassthrough`で入力と出力を確認してみる

In [4]:
json = {
    "input": "日本の首都は？"
}
resp = requests.post(f"{base_url}/passthrough/invoke", json=json)
resp.json()["output"]

'日本の首都は？'

In [5]:
json = {
    "input": {
        "question": "日本の首都は？"
    }
}
resp = requests.post(f"{base_url}/passthrough/invoke", json=json)
resp.json()["output"]

{'question': '日本の首都は？'}

In [6]:
json = {
    "input": [ "日本の首都は？" ]
}
resp = requests.post(f"{base_url}/passthrough/invoke", json=json)
resp.json()["output"]

['日本の首都は？']

In [7]:
resp = requests.post(f"{base_url}/passthrough/stream", json={ "input": "日本の首都は？" }, stream=True)
[line for line in resp.iter_lines(decode_unicode=True)]

['event: metadata',
 'data: {"run_id": "a6e617eb-34e7-4491-8653-b73f30fdd708"}',
 '',
 'event: data',
 'data: "日本の首都は？"',
 '',
 'event: end',
 '']

## `RunnableLambda`

In [8]:
json = {
    "input": {
        "text": "abc",
        "times": 3,
    }
}
resp = requests.post(f"{base_url}/lambda/invoke", json=json)
resp.json()["output"]

'abcabcabc'

In [9]:
json = {
    "input": {
        "text": "abc",
        "times": 3,
    }
}
resp = requests.post(f"{base_url}/lambda/stream", json=json, stream=True)
[line for line in resp.iter_lines(decode_unicode=True)]

['event: metadata',
 'data: {"run_id": "d6b1df50-3e27-4826-af62-b4ec37bfae13"}',
 '',
 'event: data',
 'data: "abcabcabc"',
 '',
 'event: end',
 '']

## `RunnableGenerator`でServer-Sent Events

In [10]:
json = {
    "input": {
        "text": "hello",
        "sleep": 1,
    }
}
resp = requests.post(f"{base_url}/generator/stream", json=json, stream=True)
date = datetime.now()
for line in resp.iter_lines(decode_unicode=True):
    td = datetime.now() - date
    print((f"{td.seconds}", line))


('0', 'event: metadata')
('0', 'data: {"run_id": "b1af8c93-7033-457b-9c27-bf2fd9d893dd"}')
('0', '')
('0', 'event: data')
('0', 'data: "h"')
('0', '')
('1', 'event: data')
('1', 'data: "e"')
('1', '')
('2', 'event: data')
('2', 'data: "l"')
('2', '')
('3', 'event: data')
('3', 'data: "l"')
('3', '')
('4', 'event: data')
('4', 'data: "o"')
('4', '')
('5', 'event: end')
('5', '')


In [11]:
json = {
    "input": {
        "text": "hello",
        "sleep": 1,
    }
}
date = datetime.now()
resp = requests.post(f"{base_url}/generator/invoke", json=json, stream=True)
dt = datetime.now() - date
dt, resp.json()

(datetime.timedelta(seconds=5, microseconds=17855),
 {'output': 'hello',
  'callback_events': [],
  'metadata': {'run_id': 'cc42f55e-fa2a-4c98-ac95-9b63a810cd2c'}})

## チェーンしてみる

In [12]:
json = {
    "input": "hello",
}
resp = requests.post(f"{base_url}/chain1/stream", json=json, stream=True)
date = datetime.now()
for line in resp.iter_lines(decode_unicode=True):
    td = datetime.now() - date
    print((f"{td.seconds}", line))


('1', 'event: metadata')
('1', 'data: {"run_id": "c420fc1a-dc54-4acb-8fba-4c448c890a93"}')
('1', '')
('1', 'event: data')
('1', 'data: {"bar":"barhello1"}')
('1', '')
('2', 'event: data')
('2', 'data: {"foo":"foohello1"}')
('2', '')
('3', 'event: data')
('3', 'data: {"bar":"barhello2"}')
('3', '')
('3', 'event: data')
('3', 'data: {"foo":"foohello2"}')
('3', '')
('4', 'event: data')
('4', 'data: {"bar":"barhello3"}')
('4', '')
('4', 'event: data')
('4', 'data: {"foo":"foohello3"}')
('4', '')
('4', 'event: data')
('4', 'data: {"foobar":"foobar"}')
('4', '')
('4', 'event: end')
('4', '')


### 会話する

In [13]:
session_id = "e4d63ba5-627f-4854-b53e-5d7a83926319"
json = {
    "input": {
        "question": "日本の首都は？",
    },
    "config": {
        "configurable": {
            "session_id": session_id,
        },
    },
}
resp = requests.post(f"{base_url}/chat2/invoke", json=json)
resp.json()

{'output': '東京です。',
 'callback_events': [],
 'metadata': {'run_id': '2a4b5673-5385-4e5b-917d-ae4999e417d8'}}

In [14]:
json = {
    "input": {
        "question": "イタリアは？",
    },
    "config": {
        "configurable": {
            "session_id": session_id,
        },
    },
}
resp = requests.post(f"{base_url}/chat2/invoke", json=json)
resp.json()

{'output': 'ローマです。',
 'callback_events': [],
 'metadata': {'run_id': 'a2df4e95-2566-418b-8273-05ff41e3c056'}}

In [15]:
json = {
    "input": {
        "question": "フランスは？",
    },
    "config": {
        "configurable": {
            "session_id": session_id,
        },
    },
}
resp = requests.post(f"{base_url}/chat2/invoke", json=json)
resp.json()

{'output': 'パリです。',
 'callback_events': [],
 'metadata': {'run_id': '9d530ea5-1d47-4ec8-8557-7aea507034d7'}}